## Automated ML

Import dependencies. 

In [1]:
import os
import pandas as pd
import numpy as np
import json
import requests
import joblib
from sklearn.metrics import confusion_matrix
import itertools

from azureml.core import Dataset, Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.train.automl import AutoMLConfig

from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model
from azureml.core.environment import Environment

%conda remove xgboost
%pip install xgboost==0.90
import xgboost
#print(xgboost._version_)


Solving environment: failed

PackagesNotFoundError: The following packages are missing from the target environment:
  - xgboost



Note: you may need to restart the kernel to use updated packages.
     |████████████████████████████████| 142.8 MB 25 kB/s s eta 0:00:01B 6.3 MB/s eta 0:00:16
  Attempting uninstall: xgboost
    Found existing installation: xgboost 1.3.3
    Uninstalling xgboost-1.3.3:
      Successfully uninstalled xgboost-1.3.3
Note: you may need to restart the kernel to use updated packages.


In [2]:
print(xgboost.__version__)

0.90


## Dataset

### Overview

**Dataset:** Davide Chicco, Giuseppe Jurman: "Machine learning can predict survival of patients with heart failure from serum creatinine and ejection fraction alone". BMC Medical Informatics and Decision Making 20, 16 (2020)

Heart failure is a common event caused by CVDs and this dataset contains 12 features that can be used to predict mortality by heart failure.

**12 clinical features:**

- age: age of the patient (years)
- anaemia: decrease of red blood cells or hemoglobin (boolean)
- high blood pressure: if the patient has hypertension (boolean)
- creatinine phosphokinase (CPK): level of the CPK enzyme in the blood (mcg/L)
- diabetes: if the patient has diabetes (boolean)
- ejection fraction: percentage of blood leaving the heart at each contraction (percentage)
- platelets: platelets in the blood (kiloplatelets/mL)
- sex: woman or man (binary)
- serum creatinine: level of serum creatinine in the blood (mg/dL)
- serum sodium: level of serum sodium in the blood (mEq/L)
- smoking: if the patient smokes or not (boolean)
- time: follow-up period (days)

In this project, Azure AutoML will be used to make prediction on the death event based on patient's 12 clinical features. 

## Workspace setup

In [3]:
ws = Workspace.from_config()

experiment_name = 'automl-experiment'

experiment = Experiment(ws, experiment_name)

In [4]:
dataset = Dataset.get_by_name(ws, 'heart-disease-kaggle')

In [5]:
dataset.take(5).to_pandas_dataframe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


In [7]:
data_train, data_test = dataset.random_split(0.9)

## Config Compute Cluster

In [8]:
cpu_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D12_V2',
                                                           max_nodes=5)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Creating......
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## AutoML Configuration

In [9]:
automl_settings = {
    "experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 4,
    "primary_metric" : 'accuracy',
    "n_cross_validations": 5
}
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=data_train,
                             label_column_name="DEATH_EVENT", 
                             enable_early_stopping= True,
                             featurization= 'auto',
                             **automl_settings
                            )

AutoML typically performs cross validation, data balance check, cardinality check in prior to machine learning process with a variety of algorithms. 

In [10]:
remote_run = experiment.submit(automl_config, show_output = True)

Running on remote.
No run_configuration provided, running on cpu-cluster with default configuration
Running on remote compute: cpu-cluster
Parent Run ID: AutoML_4d9955d1-6df0-4f9c-adef-883d4a0758ec

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values we

## Run Details

In [11]:
remote_run

Experiment,Id,Type,Status,Details Page,Docs Page
automl-experiment,AutoML_4d9955d1-6df0-4f9c-adef-883d4a0758ec,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [12]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [13]:
remote_run.wait_for_completion()

{'runId': 'AutoML_4d9955d1-6df0-4f9c-adef-883d4a0758ec',
 'target': 'cpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-03-24T21:48:25.782499Z',
 'endTimeUtc': '2021-03-24T22:38:27.020385Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'cpu-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"automl-experiment","subscription_id":"d7f39349-a66b-446e-aba6-0053c2cf1c11","resource_group":"aml-quickstarts-141302","workspace_name":"quick-starts-ws-141302","region":"southcentralus","compute_target":"cpu-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":5,"y_min":null,"y_ma

## Best Model 

In [14]:
best_run, fitted_model = remote_run.get_output()
best_run_metrics = best_run.get_metrics()

In [15]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
automl-experiment,AutoML_4d9955d1-6df0-4f9c-adef-883d4a0758ec_36,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [16]:
fitted_model

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                random_state=None,
                                                                                                reg_alpha=0.47368421052631576,
                                                                                                reg_lambda=0.15789473684210525,
                      

In [17]:
print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['accuracy'])
print(fitted_model._final_estimator)
print(best_run.get_tags())

Best Run Id:  AutoML_4d9955d1-6df0-4f9c-adef-883d4a0758ec_36

 Accuracy: 0.8674165457184326
PreFittedSoftVotingClassifier(classification_labels=None,
                              estimators=[('20',
                                           Pipeline(memory=None,
                                                    steps=[('standardscalerwrapper',
                                                            <azureml.automl.runtime.shared.model_wrappers.StandardScalerWrapper object at 0x7f53049a1be0>),
                                                           ('lightgbmclassifier',
                                                            LightGBMClassifier(boosting_type='gbdt',
                                                                               class_weight=None,
                                                                               colsample_bytree=0.6933333333333332,
                                                                               importance_typ...
 

In [18]:
os.makedirs('./outputs', exist_ok=True)

joblib.dump(fitted_model, filename='outputs/automl.joblib')

model_name = best_run.properties['model_name']
model_name

'AutoML4d9955d1636'

In [19]:
from azureml.automl.core.shared import constants
env = best_run.get_environment()

script_file = 'score.py'

best_run.download_file('outputs/scoring_file_v_1_0_0.py', script_file)

best_run.download_file(constants.CONDA_ENV_FILE_PATH, 'env.yml')

## Model Deployment

In [20]:
#Register the fitted model
model = remote_run.register_model(model_name = model_name,
                                  description = 'AutoML model')

In [22]:
inference_config = InferenceConfig(entry_script = script_file, environment = env)

aci_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1, enable_app_insights = True, auth_enabled = True)
#description = 'predcition if the person is likely to die'
aci_service_name = 'automl-heart-disease'
print(aci_service_name)

automl-heart-disease


In [23]:
service = Model.deploy(ws, aci_service_name, [model], inference_config, aci_config)
service.wait_for_deployment(True)
print("State: " + service.state)
print("Scoring URI: " + service.scoring_uri)
print("Keys: " + service.get_keys()[0])
print("Swagger URI: " + service.swagger_uri)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-03-24 23:01:53+00:00 Creating Container Registry if not exists.
2021-03-24 23:01:54+00:00 Use the existing image.
2021-03-24 23:01:55+00:00 Generating deployment configuration.
2021-03-24 23:01:56+00:00 Submitting deployment to compute..
2021-03-24 23:01:59+00:00 Checking the status of deployment automl-heart-disease..
2021-03-24 23:02:49+00:00 Checking the status of inference endpoint automl-heart-disease.
Succeeded
ACI service creation operation finished, operation "Succeeded"
State: Healthy
Scoring URI: http://332fefcb-8ec5-44d5-98cf-36b8099e3115.southcentralus.azurecontainer.io/score
Keys: G9YZdyl67xwpiAa6c8GEoB0sYAQxrPmL
Swagger URI: http://332fefcb-8ec5-44d5-98cf-36b8099e3115.southcentralus.azurecontainer.io/swagger.json


## Testing using 2 different methods

In [24]:
%run endpoint.py

{"result": [1, 1]}


In [25]:
data_test = data_test.to_pandas_dataframe().dropna()
data_sample = data_test.sample(3)
y_true = data_sample.pop('DEATH_EVENT')
sample_json = json.dumps({'data':data_sample.to_dict(orient='records')})
print(sample_json)

{"data": [{"age": 65.0, "anaemia": 0, "creatinine_phosphokinase": 326, "diabetes": 0, "ejection_fraction": 38, "high_blood_pressure": 0, "platelets": 294000.0, "serum_creatinine": 1.7, "serum_sodium": 139, "sex": 0, "smoking": 0, "time": 220}, {"age": 60.0, "anaemia": 1, "creatinine_phosphokinase": 95, "diabetes": 0, "ejection_fraction": 60, "high_blood_pressure": 0, "platelets": 337000.0, "serum_creatinine": 1.0, "serum_sodium": 138, "sex": 1, "smoking": 1, "time": 146}, {"age": 42.0, "anaemia": 1, "creatinine_phosphokinase": 250, "diabetes": 1, "ejection_fraction": 15, "high_blood_pressure": 0, "platelets": 213000.0, "serum_creatinine": 1.3, "serum_sodium": 136, "sex": 0, "smoking": 0, "time": 65}]}


In [26]:
output = service.run(sample_json)
print('Prediction: ', output)
print('True Values: ', y_true.values)

Prediction:  {"result": [0, 0, 1]}
True Values:  [0 0 1]


In [27]:
service.get_logs()

'2021-03-24T23:02:47,013990800+00:00 - gunicorn/run \n2021-03-24T23:02:47,033371000+00:00 - iot-server/run \n2021-03-24T23:02:47,024359600+00:00 - rsyslog/run \n2021-03-24T23:02:47,057951800+00:00 - nginx/run \n/usr/sbin/nginx: /azureml-envs/azureml_2b14f450572e78de640d54eaabed5e4d/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_2b14f450572e78de640d54eaabed5e4d/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_2b14f450572e78de640d54eaabed5e4d/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_2b14f450572e78de640d54eaabed5e4d/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_2b14f450572e78de640d54eaabed5e4d/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)

In [29]:
service.delete()